In [1]:
import requests
import pandas as pd
import json
import time
import funcoes
from datetime import datetime
# configs
pd.set_option("display.date_dayfirst", True)
# variaveis globais
tks = ['HihrPwM6iplpMc20YafyI0X20hYBwMJjn9O5b3uRXLc%3D','NHvW6AeyLM3HFKp1HwXQIbDbGVdDOyMhvbyHgEoZY57S4pEFqdDDKy1R%2FVQHplHm','UVzI3W1fv8mAK28sWaTc%2BOlAefhF1q26WWTfaOmb%2BSY6F1b2IE43cgRMrafYMySh','oAlfna7l2QNiS6nZo4oIQSQCpNo5XX9iydYqEGfMVkDVz8fBG2Vl6bW9lChTtGtP', 'vcMSbDWqyMNu8fcVLol8l0dZ0DZhinIFKyydHCBSAb%2B5RloWjteXNY9345MIcbUQ']
metodo = 'GET'
url = "https://elysia.zeev.it"
tipoUrl = "/api/2/assignments"
filtro = {'flowId': 0}
today = datetime.today().strftime('%d/%m/%Y')

In [89]:
# Se a API alterar seu campos novamente, sera necessario alterar apenas alguns filtros desta funcao
# sem gerar problemas maiores para refatorar este codigo
def request_Orquestra(method='GET',urlReq="https://elysia.zeev.it",typeReq="/api/2/assignments",tokens={},filters={}):
        retorno = []
        try:
                # filtros base para o payload
                filters['recordsPerPage'] = 100
                filters['mobileEnabledOnly'] = False
                
                # variaveis de controle
                row = 1
                i = 1
                # faz a busca para cada token
                for tk in tokens:
                        # reseta variaveis de controle
                        row = 1
                        i = 1
                        # tokens de acesso dos usuarios que queremos buscar as atribuicoes
                        authorization= {'Authorization': "Bearer " + tk}
                        while row != 0:
                                # aguardo 1s devido ao limite de requisicoes por segundo
                                time.sleep(1) 
                                
                                # incremento de pagina
                                filters['pageNumber'] = i
                                
                                # requisicao
                                getReq = funcoes.requests_API_Orquestra(metodo=method,urlAcesso=urlReq,tipoAcesso=typeReq,head=authorization,payload=filters)
                                
                                # resultado json
                                result = getReq.json()
                                        
                                # checa se esta vazio o resultado, o que significa que nao tem mais paginas
                                if (len(result) == 0) or (getReq.status_code != 200):
                                        # log
                                        break
                                else:
                                        # coloca cada retorno na lista
                                        retorno.append(result)
                                        i+=1
                # cada usuario do token fica em uma uma sublista da lista de retorno
                # por isto a necessidade de explodir ela dentro de outra lista
                retornoLista = [item for sublista in retorno for item in sublista]
                # log
                return retornoLista 
        except Exception as e:
                # log
                funcoes.printError(e)
                return [] 

In [4]:
# funcao que pega o retorno verifica se eh valido, caso nao seja retornar erro, se for valido formata as datas e começa a contagem de cada 
# verifica se o retorno eh uma lista 
def formata_DataFrame(lista):
    # verifica se houve retorno
    try:
        # ocorreu erro ou nao ocorreram retornos - provisorio
        if len(lista) <= 0:
            return -1
        # houve retorno de linhas
        elif len(lista) > 0:
            df = pd.DataFrame(lista)
            # Colocando o idTask para conferencia
            df['idTask'] = pd.DataFrame(df['instance'].apply(pd.Series))['id']
            # selecionando apenas as colunas necessarias
            df = df[['idTask','id','taskName','requestName','startDateTime','expirationDateTime','flow','instance']]

            # convertendo colunas para datetime
            df['expirationDateTime'] = pd.to_datetime(df['expirationDateTime'], dayfirst=True)
            df['startDateTime'] = pd.to_datetime(df['startDateTime'], dayfirst=True)

            # formatando data em DD/MM/YY
            df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%d/%m/%Y')
            df.loc[:,'startDateTime'] = df.loc[:,'startDateTime'].dt.strftime('%d/%m/%Y')

            # remove duplicatas de tarefas
            print('Contagem de tarefas \"ANTES\" da remocao das tarefas duplicatas: {0}'.format(str(len(df))))
            df = df.drop_duplicates(subset=['idTask'])
            print('Contagem de tarefas \"DEPOIS\" da remocao das tarefas duplicatas: {0}'.format(str(len(df))))

            # adiciona coluna de tarefa atrasada
            df['taskAtrasada'] = False
            df.loc[today > df['expirationDateTime'],['taskAtrasada']] = True
            
            # tudo ok, retorna o data frame
            return df
    except Exception as e:
        funcoes.printError(e)
        return -1


In [128]:
# Para testes
lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
df = formata_DataFrame(lista)

---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
Contagem de tarefas "ANTES" da remocao das tarefas duplicatas: 284
Contagem de tarefas "DEPOIS" da remocao das tarefas duplicatas: 253


C:\Users\Lukas\AppData\Local\Temp\ipykernel_15040\1119755624.py:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%d/%m/%Y')
C:\Users\Lukas\AppData\Local\Temp\ipykernel_15040\1119755624.py:22: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%d/%m/%Y')
C:\Users\Lukas\AppData\Local\Temp\ipykernel_15040\1119755624.py:23: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format 

In [135]:
def main_Func():
    # requisicao com todos os dados iniciais
    lista = request_Orquestra(method=metodo,urlReq=url,typeReq=tipoUrl,tokens=tks)
    df = formata_DataFrame(lista)

    # CS1 Abertos sao os Efetivos Em espera + Em Aberto nas atribuicoes de cada usuario 
    cs1 = df.loc[(df['requestName'].str.lower().str.replace(" ","").str.contains('projetov.20')) & (df['taskName'].str.lower().str.contains('ligação do cs'))]
    contCs1Abertos = len(cs1) # FALAR COM REQUISITANTE
    # CS1 Efetivos sao os Em espera do 'Servico Orquestra'
    # ------
    # CS1 Atrasados
    contCS1Atrasadas = len(cs1.loc[df['taskAtrasada'] == True])

    # CS2 Em aberto
    cs2 = df.loc[(df['requestName'].str.lower().str.contains('degustação')) | (df['requestName'].str.lower().str.contains('normal')) & ~(df['taskName'].str.lower().str.contains('relatório')) | (df['taskName'].str.lower().str.contains('suporte'))]
    contCs2Abertos = len(cs2)
    # CS2 Efetivos
    cs2Efetivo = cs2.loc[~(cs2['taskName'].str.lower().str.contains('ligação do cs'))]
    contCs2Efetivos = len(cs2Efetivo)
    # CS2 Atrasados
    cs2Atrasados = cs2.loc[(cs2['taskName'].str.lower().str.contains('ligação do cs')) & (cs2['taskAtrasada'] == True)]
    contCs2Atrasados = len(cs2Atrasados)

    # Acompanhamento Personalizado
    contClientesfluxoNormal = len(df.loc[df['requestName'].str.lower().str.contains('normal')])

    # Envio de Layout Pendentes   
    envLayout = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Baixar Layout e apresentar ao cliente')]
    contEnvLayoutPendentes = len(envLayout)
    # Envio de Layout Atrasados
    contEnvLayoutAtrasadas = len(envLayout[(envLayout['taskName'] == 'Baixar Layout e apresentar ao cliente') & (envLayout['taskAtrasada'] == True)])

    # Envio de Pendentes
    envDoc = df.loc[(df['requestName'] == 'Projeto  v. 20') & (df['taskName'] == 'Entregar Documentação ao Cliente')]
    envDocPendentes = len(envDoc)
    # Envio de Documentos Atrasados
    envDocAtrasados = len(envDoc.loc[envDoc['taskAtrasada'] == True])
    # Print
    print('\n-------------------------------------------------------------------------------------------------------------')
    print('Contatos Em Aberto CS1 - {0}\nContatos Efetivos CS1 - \nContatos Em Atraso CS1 - {1}'.format(contCs1Abertos,contCS1Atrasadas)) # falta aberto e efetivo
    print('Contatos Em Aberto CS2 - {0}\nContatos Efetivos CS2 - {1}\nContatos Em Atraso CS2 - {2}'.format(contCs2Abertos, contCs2Efetivos,contCs2Atrasados)) # concluido
    print('Acompanhamento Personalizado - {0}'.format(contClientesfluxoNormal)) # concluido
    print('Envio de Layouts Pendentes  - {0}\nEnvio de Layouts Realizados - \nEnvio de Layouts Atrasados  - {1}'.format(contEnvLayoutPendentes, contEnvLayoutAtrasadas)) # falta realizados
    print('Envio de Documentos Pendentes - {0}\nEnvio de Documentos Efetivos  - \nEnvio de Documentos Atrasados - {1}'.format(envDocPendentes, envDocAtrasados)) # falta efetivos
    print('---------------------------------------------------------------------------------------------------------------')
main_Func()

---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
---> Tipo de requisicao: 'GET'
Contagem de tarefas "ANTES" da remocao das tarefas duplicatas: 284
Contagem de tarefas "DEPOIS" da remocao das tarefas duplicatas: 253

-------------------------------------------------------------------------------------------------------------
Contatos Em Aberto CS1 - 7
Contatos Efetivos CS1 - 
Contatos Em Atraso CS1 - 7
Contatos Em Aberto CS2 - 103
Contatos Efetivos CS2 - 43
Contatos Em Atraso CS2 - 58
Acompanhamento Personalizado - 19
Envio de Layouts Pendentes  - 1
Envio de Layouts Realizados - 
Envio de Layouts Atrasados  - 1
Envio de Documentos Pendentes - 0
Envio de Documentos Efetivos  - 
Envio de Documentos Atrasados - 0
--------------------

C:\Users\Lukas\AppData\Local\Temp\ipykernel_15040\1119755624.py:22: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%d/%m/%Y')
C:\Users\Lukas\AppData\Local\Temp\ipykernel_15040\1119755624.py:22: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'expirationDateTime'] = df.loc[:,'expirationDateTime'].dt.strftime('%d/%m/%Y')
C:\Users\Lukas\AppData\Local\Temp\ipykernel_15040\1119755624.py:23: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format 

# CS1 
* **Contato Em Aberto CS1** - Após a troca do relógio -> CS - DATA DA TROCA DO MEDIDOR - PREENCHIDO
  APÓS SER FINALIZADA A TAREFA SAI DO CONTATO EM ABERTO E O CAMPO DE  STATUS CLIENTE FICA COMO: FINALIZADO
  *  *OBS: No entanto, a tarefa fica ‘fechada’ temporariamente, então é necessário verificar as tarefas ‘fechadas’ para contabilizar.*
* **Contato Efetivo CS1** - STATUS CLIENTE - EM ABERTO - **OK**
* **Contato Atrasado CS1**- Apenas os atrasados - **OK**

# CS2
* **Contato em Aberto CS2** - Contar Degustação e Normal - **OK**
    * Verificar a localização do fluxo deles;
* **Contato Efetivo CS2** - Mesmo procedimento do CS1 **OK**
* **Contato Atrasados** - Mesmo procedimento CS1 - **OK**

# Acompanhamento Personalizado 
* **Nro de Clientes no Fluxo Normal**

# Envio de Documentos
* **Env. Doc. Pendentes**  - Tarefas do Reginatto -> Envio Documentos Aberto - **OK**
* **Env. Doc. Realizados** - Tarefas do Reginatto -> Envio Documentos Fechado
* **Env. Doc. Atrasados**  - Tarefas do Reginatto -> Envio Documentos com Atraso - **OK**

# Envios Layout - 'Baixar Layout e apresentar ao cliente'
* **Envios Layout Pendentes**    - Tarefas do Reginatto -> Baixar Layouts Aberto   - **OK**
* **Envios Layout  Realizados**  - Tarefas do Reginatto -> Baixar Layouts Fechado
* **Envios Layout Atrasados**    - Tarefas do Reginatto -> Baixar Layouts Atrasado - **OK**